<a href="https://colab.research.google.com/github/jeevan-nh/Google-collab-udemy-DL-course/blob/master/Boltzman's_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BOLTZMAN MACHINE 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

##DATA IMPORTING AND PREPROCESSING

In [0]:
#importing the dataset
movies = pd.read_csv('/content/drive/My Drive/Colab Notebooks/P16-Boltzmann-Machines/Boltzmann_Machines/ml-1m/movies.dat',sep='::', 
                     header=None,engine = 'python',encoding = 'latin-1')

In [6]:
movies.head(10)

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [8]:
users = pd.read_csv('/content/drive/My Drive/Colab Notebooks/P16-Boltzmann-Machines/Boltzmann_Machines/ml-1m/users.dat',sep='::', 
                     header=None,engine = 'python',encoding = 'latin-1')
users.head(10)

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


In [9]:
ratings = pd.read_csv('/content/drive/My Drive/Colab Notebooks/P16-Boltzmann-Machines/Boltzmann_Machines/ml-1m/ratings.dat',sep='::', 
                     header=None,engine = 'python',encoding = 'latin-1')
ratings.head(10)

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [0]:
#preparing training and test set
training_set = pd.read_csv('/content/drive/My Drive/Colab Notebooks/P16-Boltzmann-Machines/Boltzmann_Machines/ml-100k/u1.base',delimiter='\t')
training_set = np.array(training_set, dtype = 'int') #converting to array
test_set = pd.read_csv('/content/drive/My Drive/Colab Notebooks/P16-Boltzmann-Machines/Boltzmann_Machines/ml-100k/u1.test',delimiter='\t')
test_set = np.array(test_set, dtype = 'int') #converting to array

In [14]:
#Getting the number of users and movies 
nb_users = int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]),max(test_set[:,1])))
print(f'max no of user --> {nb_users}\nmax no of movies --> {nb_movies}')

max no of user --> 943
max no of movies --> 1682


In [0]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
  new_data = []
  for id_users in range(1,nb_users + 1):
    id_movies = data[:,1][data[:,0] == id_users]
    id_ratings = data[:,2][data[:,0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [0]:
#converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

##WILL USERS LIKE A MOVIE OR NOT ??

In [0]:
#converting the Rating into binary Rating
#in pytorch OR/AND condition will not work
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

###Creating the NN 

In [0]:
#creating the architecture of NN
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [0]:
#creating the RBM object
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

In [29]:
#training of the model
nb_epoch  = 10
for epoch in range(1,nb_epoch + 1):
  train_loss = 0
  s = 0. 
  for id_user in range(0,nb_users - batch_size,batch_size):
    vk = training_set[id_user:id_user + batch_size]
    v0 = training_set[id_user:id_user + batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0] #getting the -1 rating 
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0,vk,ph0,phk)
    train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
    s += 1. 
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

RuntimeError: ignored

In [0]:
# Testing the RBM
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))